In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install chromadb
!pip install --upgrade --quiet  docx2txt
!pip install langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install -q -U git+https://github.com/huggingface/transformers.git

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader, Docx2txtLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import os
from transformers import pipeline

In [3]:
folder_path = "/kaggle/input/irtm-project-2-dataset"

In [4]:
def load_documents(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith(".txt"):
            loader = TextLoader(file_path)
            txt = loader.load()
            documents.extend(txt)
        elif filename.endswith(".doc") or filename.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
            doc = loader.load()
            documents.extend(doc)
        elif filename.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
            pdf = loader.load()
            documents.extend(pdf)

    # This is where data chunking is realized
    splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=40)
    documents_split = splitter.split_documents(documents)
    
    return documents_split

In [140]:
def preprocess_documents(documents):
    # We replace unusual characters with their normal counterpart
    for i in range(len(documents)):
        documents[i].page_content = documents[i].page_content.replace('“', '"').replace('”', '"')
        documents[i].page_content = documents[i].page_content.replace('ş', 'ș').replace("Ş", "Ș").replace('ţ', 'ț').replace("Ţ", "Ț")

    return documents

In [6]:
documents = load_documents(folder_path)
documents = preprocess_documents(documents)

In [36]:
def initialize_vector_db(docs):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    chroma_db = Chroma.from_documents(persist_directory="db", documents=docs, embedding=embeddings, collection_metadata={"hnsw:space": "cosine"})

    return chroma_db

In [ ]:
chroma_db = initialize_vector_db(documents)

In [ ]:
hf_pipeline = pipeline("text-generation", model="/kaggle/input/openllm-ro-q4b/transformers/7b-instruct-q4b/1/RoLlama2-7b-Instruct-v1-q4b", kwargs=['_load_in_4bit'])

In [238]:
def get_query_context(query):
    # The IR component
    documents = chroma_db.similarity_search_with_score(query, k=15)

    print(f"{len(documents)} context documents")
    for d in documents:
        print(f"------------------ similarity (less is better) {round(d[1], 2)}")
        print(f"document: {d[0].metadata['source'].split('/')[4]} || context: {d[0].page_content}")
    print("------------------\n")

    context = " ".join([f"document: {doc[0].metadata['source'].split('/')[4]} || context: {doc[0].page_content}" for doc in documents])

    return context

In [270]:
def get_prompt_rag(query, context):
    prompt = f"""
    ---Instrucțiuni---:
    Scrie un răspuns prietenos și ușor de înțeles pentru întrebarea din cadrul câmpului ---Întrebare---:
    Genereaza doar informațiile aferente câmpului ---Răspuns---: fără să oferi context sau alte informații suplimentare
    Mai jos sunt trei exemple, ---Exemplu 1---:, ---Exemplu 2---:, și ---Exemplu 3---:, care arată cum va trebui sa răspunzi întrebării din cadrul câmpului ---Întrebare---:
    Încearcă să generezi răspunsul pe baza faptelor din câmpul ---Context---:. Dacă în datele pe care ai fost antrenat nu se află informațiile necesare pentru a răspunde la ---Întrebare---: atunci generează mesajul "Nu pot oferi un răspuns concret."
    
    ---Exemplu 1---:
    Întrebare: Ce este schimbarea climatică?
    Răspuns: Schimbarea climatică este atunci când vremea planetei noastre se modifică pe o perioadă lungă, de obicei din cauza activităților umane, cum ar fi arderea combustibililor fosili.
    
    ---Exemplu 2---:
    Întrebare: Care este capitala României?
    Răspuns: Capitala României este București.
    
    ---Exemplu 3---:
    Întrebare: Cum funcționează energia solară?
    Răspuns: Energia solară funcționează prin captarea luminii de la soare folosind panouri solare. Aceste panouri transformă lumina în electricitate pe care o putem folosi acasă.
    
    ---Context---:
    {context}
    
    ---Întrebare---: {query} Care sunt maximum top 3 documente relevante, cu nume diferite, pentru propoziția anterioară?
    ---Răspuns---: 
    """

    return prompt

In [237]:
def get_prompt_llm(query):
    prompt = f"""
    ---Instrucțiuni---:
    Scrie un răspuns prietenos și ușor de înțeles pentru întrebarea din cadrul câmpului ---Întrebare---:
    Genereaza doar informațiile aferente câmpului ---Răspuns---: fără să oferi context sau alte informații suplimentare
    Mai jos sunt trei exemple, ---Exemplu 1---:, ---Exemplu 2---:, și ---Exemplu 3---:, care arată cum va trebui sa răspunzi întrebării din cadrul câmpului ---Întrebare---:
    Dacă în datele pe care ai fost antrenat nu se află informațiile necesare pentru a răspunde la ---Întrebare---: atunci generează mesajul "Nu pot oferi un răspuns concret."
    
    ---Exemplu 1---:
    Întrebare: Ce este schimbarea climatică?
    Răspuns: Schimbarea climatică este atunci când vremea planetei noastre se modifică pe o perioadă lungă, de obicei din cauza activităților umane, cum ar fi arderea combustibililor fosili.
    
    ---Exemplu 2---:
    Întrebare: Care este capitala României?
    Răspuns: Capitala României este București.
    
    ---Exemplu 3---:
    Întrebare: Cum funcționează energia solară?
    Răspuns: Energia solară funcționează prin captarea luminii de la soare folosind panouri solare. Aceste panouri transformă lumina în electricitate pe care o putem folosi acasă.
    
    ---Întrebare---: {query} Care sunt maximum top 3 documente relevante, cu nume diferite, pentru propoziția anterioară?
    ---Răspuns---: 
    """

    return prompt

In [251]:
def get_system_response(query, system, temperature=0.1, top_p=0.8, max_new_tokens=512):
    context = get_query_context(query)

    if system == "rag":
        prompt = get_prompt_rag(query, context)
    else:
        prompt = get_prompt_llm(query)
    
    system_output = hf_pipeline(
        prompt,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        max_new_tokens=max_new_tokens,
        return_full_text=False
    )
    response = system_output[0]['generated_text']

    return response

In [ ]:
query = "Cine este Lică Sămădăul?"

In [ ]:
# RAG

system = "rag"
response = get_system_response(query, system, max_new_tokens=128)
print(response)

In [ ]:
# LLM

system = "llm"
response = get_system_response(query, system, max_new_tokens=128)
print(response)

In [ ]:
# IR

response = get_query_context(query)
print(response)